In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import collections
import os
import time
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Load pre-processed data for training

In [2]:
path_to_preprocessed_data = "C:/Users/RLOCAL/Documents/dataMedTeam/datachallenge_jfr/pre_processed/"

In [3]:
# import time
# nber_minutes = 5*60
# timeout = time.time() + 60*nber_minutes

# while True:
#     test = 0
#     if test == nber_minutes or time.time() > timeout:
#         break
#     test = test - 1

# print("hello")

In [4]:
train_data = np.load(path_to_preprocessed_data + "train_data_wi_oversampling.npy")
train_labels = np.load(path_to_preprocessed_data + "train_labels_wi_oversampling.npy")

test_data = np.load(path_to_preprocessed_data + "test_data_wi_oversampling.npy")
test_labels = np.load(path_to_preprocessed_data + "test_labels_wi_oversampling.npy")

print("Train data shape: {}".format(train_data.shape))
print("Train label shape: {}".format(train_labels.shape))

print("Test data shape: {}".format(test_data.shape))
print("Test label shape: {}".format(test_labels.shape))

Train data shape: (1479, 691200)
Train label shape: (1479, 8)
Test data shape: (40, 691200)
Test label shape: (40, 8)


# Model1: CNN

Dealing with an image processing classication problem, a convolutional neural network (CNN) classifier can bring very promising results. Given the few number of data provided, one should use anti over-fitting techniques such as : cross-validation and dropout technique.

The mathemical formula of the 2-layer CNN implemented is exposed below:

$$
y=\textrm{softmax}(ReLU(x\ast W_1+b_1)W_2+b_2)
$$


In [6]:
path_to_model = "C:/Users/RLOCAL/Documents/dataMedTeam/datachallenge_jfr/model/"
model_name = "my_cnn_model_wi_oversampling"

assert not os.path.exists(path_to_model + model_name)

if not os.path.exists(path_to_model + model_name):
    os.makedirs(path_to_model + model_name)

In [7]:
# Define computational graph (CG)
batch_size = 40         # batch size
d = train_data.shape[1]  # data dimensionality
nc = 8                  # number of classes

# CG inputs
xin = tf.placeholder(tf.float32,[batch_size,d], name = "xin"); #print('xin=',xin,xin.get_shape())
y_label = tf.placeholder(tf.float32,[batch_size,nc], name = "y_label"); #print('y_label=',y_label,y_label.get_shape())
d = tf.placeholder(tf.float32, name= "d");

# Convolutional layer
K = 5   # size of the patch
F = 10  # number of filters
ncl = K*K*F
Wcl = tf.Variable(tf.truncated_normal([K,K,1,F], stddev=tf.sqrt(2./tf.to_float(ncl)) )); #print('Wcl=',Wcl.get_shape())
bcl = tf.Variable(tf.zeros([F])); #print('bcl=',bcl.get_shape())
x_2d = tf.reshape(xin, [-1,720,960,1]); #print('x_2d=',x_2d.get_shape())
x = tf.nn.conv2d(x_2d, Wcl, strides=[1, 1, 1, 1], padding='SAME')
x += bcl; #print('x2=',x.get_shape())

# ReLU activation
x = tf.nn.relu(x)

# Dropout
x = tf.nn.dropout(x, d)

# Fully Connected layer
nfc = 720*960*F
x = tf.reshape(x, [batch_size,-1]); #print('x3=',x.get_shape())
Wfc = tf.Variable(tf.truncated_normal([nfc,nc], stddev=tf.sqrt(6./tf.to_float(nfc+nc)) )); #print('Wfc=',Wfc.get_shape())
bfc = tf.Variable(tf.zeros([nc])); #print('bfc=',bfc.get_shape())
y = tf.matmul(x, Wfc); #print('y1=',y,y.get_shape())
y += bfc; #print('y2=',y,y.get_shape())

# Softmax
y = tf.nn.softmax(y, name="op_to_restore"); #print('y3=',y,y.get_shape())

# Loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(y), 1))
total_loss = cross_entropy

# Optimization scheme
#train_step = tf.train.GradientDescentOptimizer(0.025).minimize(total_loss)
train_step = tf.train.AdamOptimizer(0.001).minimize(total_loss)

# Accuracy
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_label,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# Run Computational Graph
n = train_data.shape[0]
indices = collections.deque()
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
sess.run(init)
start_time = time.time()
for i in range(501):
    
    # Batch extraction
    if len(indices) < batch_size:
        indices.extend(np.random.permutation(n)) 
    idx = [indices.popleft() for i in range(batch_size)]
    batch_x, batch_y = train_data[idx,:], train_labels[idx]
    #print(batch_x.shape,batch_y.shape)
    
    # Run CG for variable training
    _,acc_train,total_loss_o = sess.run([train_step,accuracy,total_loss], feed_dict={xin: batch_x, y_label: batch_y, d: 0.25})
    
    # Run CG for test set
    if not i%25:
        print('\nIteration i=',i,', train accuracy=',acc_train,', loss=',total_loss_o)
        acc_test = sess.run(accuracy, feed_dict={xin: test_data, y_label: test_labels, d: 1.0})
        print('test accuracy=',acc_test)
        print("elapsed time= {} s".format(time.time() - start_time))

saver.save(sess, path_to_model + model_name + "/" + model_name)

Instructions for updating:
Use `tf.global_variables_initializer` instead.

Iteration i= 0 , train accuracy= 0.125 , loss= 4.314842
test accuracy= 0.55
elapsed time= 11.44505763053894 s

Iteration i= 25 , train accuracy= 0.375 , loss= 4.8089175
test accuracy= 0.55
elapsed time= 199.7757704257965 s

Iteration i= 50 , train accuracy= 0.35 , loss= 4.4305625
test accuracy= 0.3
elapsed time= 384.4500787258148 s

Iteration i= 75 , train accuracy= 0.575 , loss= 3.1714473
test accuracy= 0.675
elapsed time= 567.2900023460388 s

Iteration i= 100 , train accuracy= 0.375 , loss= 2.6897457
test accuracy= 0.375
elapsed time= 749.8853580951691 s

Iteration i= 125 , train accuracy= 0.425 , loss= 2.2012868
test accuracy= 0.65
elapsed time= 929.8044137954712 s

Iteration i= 150 , train accuracy= 0.45 , loss= 2.4907663
test accuracy= 0.45
elapsed time= 1110.8596503734589 s

Iteration i= 175 , train accuracy= 0.325 , loss= 2.5246706
test accuracy= 0.4
elapsed time= 1291.7580778598785 s

Iteration i= 200 , 

'C:/Users/RLOCAL/Documents/dataMedTeam/datachallenge_jfr/model/my_cnn_model_wi_oversampling/my_cnn_model_wi_oversampling'